In [1]:
from src.utils import get_sedona, setup_logger
import geopandas as gpd
import pandas as pd
import fiona
import pyspark.sql.functions as f
from sedona.spark import *
import logging
import time


In [2]:
logger = setup_logger("EVALUATION", level=logging.DEBUG)

config_path = 'src/config.json'
sedona = get_sedona(config_path)

25/03/19 21:48:43 WARN Utils: Your hostname, cirowhois resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp0s20f3)
25/03/19 21:48:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1


:: loading settings :: url = jar:file:/home/cirowhois/CODE/etl-criminalidade-sp/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cirowhois/.ivy2/cache
The jars for the packages stored in: /home/cirowhois/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b3b0043-551d-4955-b0be-156144f26ebd;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.0 in central
	found org.apache.sedona#sedona-common;1.7.0 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.g

## Dissolve
Teste de tempo comparando GeoPandas x Sedona para dissolver o campo de município (CD_MUN) e sua respectiva geometria de um geodataset de Setores Censitários do Brasil com 472.780 geometrias complexas.

* [GeoPandas: gdf.dissolve()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.dissolve.html)
* [Sedona: ST_Union_Aggr](https://sedona.apache.org/1.5.0/api/sql/AggregateFunction/#st_union_aggr)

In [3]:
file = 'src/data/landing_area/ibge/scs/BR_setores_CD2022.gpkg'

# Sedona
def test_dissolve_sedona():
    start_time = time.time()
    # read
    data = sedona.read.format("geopackage").option("tableName", 'BR_setores_CD2022').load(file)
    # dissolve
    dissolved = data.groupBy("CD_MUN","NM_MUN", "CD_UF").agg(f.expr("ST_Union_Aggr(ST_Transform(geom,'epsg:4674','epsg:4326'))").alias("geom"))
    # write
    dissolved.write.mode('overwrite')\
                  .format('parquet')\
                  .parquet('evaluation/dissolve/sedona/')
    elapsed_time = time.time() - start_time
    logger.info(f'dissolve_sedona: {elapsed_time}s')

test_dissolve_sedona()
    
# Geopandas
def test_dissolve_geopandas():
    start_time = time.time()
    # read
    data = gpd.read_file(file)
    # dissolve
    dissolved = data.dissolve(by=["CD_MUN", "NM_MUN", "CD_UF"]).reset_index().to_crs(4326)
    # write
    dissolved[['CD_MUN','NM_MUN','CD_UF','geometry']].to_parquet('evaluation/dissolve/geopandas/mun_dissolved.parquet')
    elapsed_time = time.time() - start_time
    logger.info(f'dissolve_geopandas: {elapsed_time}s')

test_dissolve_geopandas()


25/03/19 21:49:03 WARN GeoPackageScan: Loading one large unsplittable file file:/home/cirowhois/CODE/etl-criminalidade-sp/src/data/landing_area/ibge/scs/BR_setores_CD2022.gpkg with only one partition, the reason is: undefined
25/03/19 21:49:03 WARN GeoPackageScan: Loading one large unsplittable file file:/home/cirowhois/CODE/etl-criminalidade-sp/src/data/landing_area/ibge/scs/BR_setores_CD2022.gpkg with only one partition, the reason is: undefined
25/03/19 21:49:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


2025-03-19 21:53:23 - EVALUATION - INFO - dissolve_sedona: 261.6176338195801s


2025-03-19 22:03:25 - EVALUATION - INFO - dissolve_geopandas: 601.6481568813324s
